# Computation Graph and Expressions

In [1]:
import dynet as dy

dy.renew_cg()

v1 = dy.inputVector([1,2,3,4])

v2 = dy.inputVector([5,6,7,8])

v3 = v1 + v2

v4 = v3 * 2

v5 = v1 + 1

v6 = dy.concatenate([v1,v2,v3,v5])

print(v6)

print(v6.npvalue())

expression 5/1
[ 1.  2.  3.  4.  5.  6.  7.  8.  6.  8. 10. 12.  2.  3.  4.  5.]


In [2]:
model = dy.Model()

pW = model.add_parameters((20,4))
pb = model.add_parameters(20)

dy.renew_cg()

x = dy.inputVector([1,2,3,4])
W = dy.parameter(pW)
b = dy.parameter(pb)

y = W * x + b

print(y)

print(y.npvalue())

The dy.parameter(...) call is now DEPRECATED.
        There is no longer need to explicitly add parameters to the computation graph.
        Any used parameter will be added automatically.
expression 4/2
[ 0.60428685 -3.05060434 -1.2845515   2.55419135  3.87612057  1.67413712
  1.48315549  1.10197604 -1.26364303 -1.83409572 -2.86404896  2.58289647
  2.96064782  1.62793159 -1.51615036  1.17423546  1.50631154 -0.35740501
  2.45573211  3.11065388]


In [3]:
model = dy.Model()

trainer = dy.SimpleSGDTrainer(model)

p_v = model.add_parameters(10)

EPOCHS = 10

for epoch in range (EPOCHS):
    dy.renew_cg()
    
    v = dy.parameter(p_v)
    v2 = dy.dot_product(v,v)
    v2.forward()
    
    v2.backward()
    
    trainer.update()
    
print(v2.npvalue())

[0.02144409]


In [4]:
# XOR problem

import dynet as dy
import random

data = [([0,1],0),
        ([1,0],0),
        ([0,0],1),
        ([1,1],1)]

model = dy.Model()
pU = model.add_parameters((4,2))
pb = model.add_parameters(4)
pv = model.add_parameters(4)

trainer = dy.SimpleSGDTrainer(model)




EPOCHS = 1000

for epoch in range (EPOCHS):
    epoch_loss = 0.0
    for x,y in data:
        dy.renew_cg()
        U = dy.parameter(pU)
        b = dy.parameter(pb)
        v = dy.parameter(pv)
        x = dy.inputVector(x)
        
        # prediction
        
        yhat = dy.logistic(dy.dot_product(v,dy.tanh(U * x + b)))
        
        # calculate loss
        if y == 0:
            loss = -dy.log(1 - yhat)
        else:
            loss = -dy.log(yhat)
            
        epoch_loss += loss.scalar_value()
        
        loss.backward()
        trainer.update()
        
    print("Epoch %d. loss = %f" % (epoch, epoch_loss))
    
inputData = [0,0]
i = dy.inputVector(inputData)
yhat = dy.logistic(dy.dot_product(v,dy.tanh(U * x + b)))

print(yhat.value())
            
        
        
        
    

Epoch 0. loss = 3.106675
Epoch 1. loss = 3.079773
Epoch 2. loss = 3.057939
Epoch 3. loss = 3.039737
Epoch 4. loss = 3.024225
Epoch 5. loss = 3.010765
Epoch 6. loss = 2.998916
Epoch 7. loss = 2.988360
Epoch 8. loss = 2.978866
Epoch 9. loss = 2.970259
Epoch 10. loss = 2.962401
Epoch 11. loss = 2.955188
Epoch 12. loss = 2.948533
Epoch 13. loss = 2.942364
Epoch 14. loss = 2.936625
Epoch 15. loss = 2.931266
Epoch 16. loss = 2.926245
Epoch 17. loss = 2.921527
Epoch 18. loss = 2.917080
Epoch 19. loss = 2.912879
Epoch 20. loss = 2.908898
Epoch 21. loss = 2.905117
Epoch 22. loss = 2.901519
Epoch 23. loss = 2.898085
Epoch 24. loss = 2.894802
Epoch 25. loss = 2.891655
Epoch 26. loss = 2.888633
Epoch 27. loss = 2.885724
Epoch 28. loss = 2.882918
Epoch 29. loss = 2.880207
Epoch 30. loss = 2.877580
Epoch 31. loss = 2.875031
Epoch 32. loss = 2.872552
Epoch 33. loss = 2.870136
Epoch 34. loss = 2.867776
Epoch 35. loss = 2.865466
Epoch 36. loss = 2.863202
Epoch 37. loss = 2.860976
Epoch 38. loss = 2.858

In [1]:
# Deep Averaging Network (Look-up table)
import dynet as dy
import random
import pandas as pd
import numpy as np
import csv

def read_dataset(fname):
    print('Reading dataset...')
    df = pd.read_csv(fname)

    tags_index = {
        "sci-fi": 1,
        "action": 2,
        "comedy": 3,
        "fantasy": 4,
        "animation": 5,
        "romance": 6,
    }
    
    index_tags = {
        1 : "sci-fi",
        2 : "action",
        3 : "comedy",
        4 : "fantasy",
        5 : "animation",
        6 : "romance",
    }
   
    data = []
    with open(fname) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                line_count += 1
            else:
                data.append([row[2],tags_index[row[3].strip("\n")]])
                line_count += 1

    return data, tags_index,index_tags

def read_glove():
    print('Loading word vectors...')
    word2vec = {}
    embedding = []
    embeds = []
    word2idx = {}
    with open('glovo/glove.6B.50d.txt', encoding='utf-8') as f:
      for line in f:
        values = line.lower().split()
        word = values[0]
        word2idx[word] = len(embeds)
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec
        embeds.append(vec)
    return np.array(embeds),word2idx
    
    
data, l2i, i2l = read_dataset("tagged_plots_movielens.csv")
embedding , w2i = read_glove()

Reading dataset...
Loading word vectors...


In [2]:

model = dy.Model()
trainer = dy.SimpleSGDTrainer(model)

l2i = {}
i2l = {}

V = 1000
HID = 20
EDIM = 50
NOUT = 5


def predict_labels(doc):
    x = encode_doc(doc)
    h= layer1(x)
    y = layer2(h)
    
    return dy.softmax(y)


def layer1(x):
    W = dy.parameter(pW1)
    b = dy.parameter(pb1)
    
    return dy.tanh(W * x + b)

def layer2(x):
    W = dy.parameter(pW2)
    b = dy.parameter(pb2)
    
    return dy.tanh(W * x + b)

def encode_doc(doc):
    doc2=[]
    embs=[]
    for w in doc:
        try:
            doc2.append(w2i[w])
        except:
            pass
    for idx in doc2:
        try:
            embs.append(E[idx])
        except:
            pass
    
#     doc = [w2i[w] for w in doc]
#     embs = [E[idx] for idx in doc]

    return dy.esum(embs)

def do_loss(probs, label):
    try:
        label = l2i[label]
    except:
#         print("pick=>",dy.pick(probs,2))
#         print("LOG--->>>",-dy.log(dy.pick(probs,2)))
        return -dy.log(dy.pick(probs,2))

    return -dy.log(dy.pick(probs,label))

def classify(doc):
    dy.renew_cg()
    probs = predict_labels(doc)
    
    vals = probs.npvalue()
    
    return i2l[np.argmax(vals)]



# HID = 20
# EDIM = 50
# NOUT = 5
pW1 = model.add_parameters((HID, EDIM))
pb1 = model.add_parameters(HID)
pW2 = model.add_parameters((NOUT, HID))
pb2 = model.add_parameters(NOUT)
E = model.add_lookup_parameters((len(w2i), EDIM),init =  embedding )
print(E)
print(E[0])
print(E[1])

# for (doc, label) in data:
#     dy.renew_cg()
#     probs = predict_labels(doc)
    
#     loss = do_loss(probs,label)
#     loss.forward()
#     loss.backward()
#     trainer.update()
print("########################")

LookupParameter /_4
expression 0/0
expression 1/0
########################


In [3]:
embedding[0]
# w2i["the"]

array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
       -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
        2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
        1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
       -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
       -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
        4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
        7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
       -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
        1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01],
      dtype=float32)

In [4]:
for (doc,label) in data:
    print(doc,"--",label)
    break

A little boy named Andy loves to be in his room, playing with his toys, especially his doll named "Woody". But, what do the toys do when Andy is not with them, they come to life. Woody believes that he has life (as a toy) good. However, he must worry about Andy's family moving, and what Woody does not know is about Andy's birthday party. Woody does not realize that Andy's mother gave him an action figure known as Buzz Lightyear, who does not believe that he is a toy, and quickly becomes Andy's new favorite toy. Woody, who is now consumed with jealousy, tries to get rid of Buzz. Then, both Woody and Buzz are now lost. They must find a way to get back to Andy before he moves without them, but they will have to pass through a ruthless toy killer, Sid Phillips. -- 5


In [ ]:

def predict_labels(doc,b,U,d,H):
    x = encode_doc(doc)
    _h= layer1(x,H,d)
    y = b+U*_h
    
    return dy.softmax(y)


def layer1(x,H,d):
    _H = dy.parameter(H)
    _d = dy.parameter(d)
    
    return dy.tanh(_H * x + _d)

def encode_doc(doc):
    doc2=[]
    embs=[]
    for w in doc:
        try:
            doc2.append(w2i[w])
        except:
            doc2.append(w2i["mmeann"])
    for idx in doc2:
        embs.append(E[idx])
    
    return dy.esum(embs)


def do_loss(probs, next_word, w2v):
    try:
        next_word_vector = w2v[next_word]
    except:
        next_word_vector = w2v["mmeann"]

    return -dy.log(dy.pick_batch_elem(probs,next_word_vector))



model = dy.Model()

trainer = dy.SimpleSGDTrainer(model)

h = 100 # HiddenUnit
m = vectorDim
OutUnit = 100

H = model.add_parameters((h, m))
d = model.add_parameters(h)
U = model.add_parameters((m, h))
b = model.add_parameters(m)

E = model.add_lookup_parameters((len(w2i), HiddenDim),init =  embedding )

for p in poems:
    for gram in p:
        dy.renew_cg()
        probs = predict_labels(gram[:ngram-1],b,U,d,H)
        
        loss = do_loss(probs,gram[-1],w2v)
        loss.forward()
        loss.backward()
        trainer.update()